In [1]:
#install plotly
import sys
!{sys.executable} -m pip install plotly

#load modules
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm

#load for visuals
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Set up notebook to display multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
import csv

In [2]:
df = pd.read_csv("E0.csv")

In [3]:
df['FTTotG'] = df['FTHG'] + df['FTAG']
df['HTTotG'] = df['HTHG'] + df['HTAG']

In [4]:
columns = ['Div', 'Referee', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 
          'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD',
          'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5',
          'Max>2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 
          'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA',
          'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH',
          'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH',
          'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC<2.5',
          'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH',
          'MaxCAHA', 'AvgCAHH', 'AvgCAHA', 'MaxC>2.5', 'Max<2.5', 'B365H', 'B365D', 'B365A']
df = df.drop(columns=columns)

In [5]:
df.loc[df['FTHG'] == df['FTAG'], 'home_result'] = 'D'
df.loc[df['FTHG'] > df['FTAG'], 'home_result'] = 'W'
df.loc[df['FTHG'] < df['FTAG'], 'home_result'] = 'L'
df.loc[df['FTHG'] == df['FTAG'], 'away_result'] = 'D'
df.loc[df['FTHG'] > df['FTAG'], 'away_result'] = 'L'
df.loc[df['FTHG'] < df['FTAG'], 'away_result'] = 'W'


In [6]:
home_team_stats = df[[
    'Date',
    'HomeTeam',
    'FTHG',
    'HTHG',
    'HS',
    'HST',
    'HF',
    'HC',
    'HY',
    'HR',
    'home_result',]]

away_team_stats = df[[
    'Date',
    'AwayTeam',
    'FTAG',
    'HTAG',
    'AS',
    'AST',
    'AF',
    'AC',
    'AY',
    'AR',
    'away_result',]]

In [7]:
home_team_stats = home_team_stats.rename(columns={'HomeTeam':'team', 'FTHG':'total_goals', 'HTHG':'half_time_goals', 'HS':'shots', 'HST':'shots_on_target',
                               'HF':'fouls', 'HC':'corners', 'HY':'yellows', 'HR':'reds'})
home_team_stats.head()

away_team_stats = away_team_stats.rename(columns={'AwayTeam':'team', 'FTAG':'total_goals', 'HTAG':'half_time_goals', 'AS':'shots', 'AST':'shots_on_target',
                               'AF':'fouls', 'AC':'corners', 'AY':'yellows', 'AR':'reds'})
away_team_stats.head()

,Date,team,total_goals,half_time_goals,shots,shots_on_target,fouls,corners,yellows,reds,home_result
0,5/8/2022,Crystal Palace,0,0,10,2,16,3,1,0,L
1,6/8/2022,Fulham,2,1,9,3,7,4,2,0,D
2,6/8/2022,Bournemouth,2,1,7,3,18,5,3,0,W
3,6/8/2022,Leeds,2,1,12,4,13,6,2,0,W
4,6/8/2022,Newcastle,2,0,23,10,9,11,0,0,W


,Date,team,total_goals,half_time_goals,shots,shots_on_target,fouls,corners,yellows,reds,away_result
0,5/8/2022,Arsenal,2,1,10,2,11,5,2,0,W
1,6/8/2022,Liverpool,2,0,11,4,9,4,0,0,D
2,6/8/2022,Aston Villa,0,0,15,2,16,5,3,0,L
3,6/8/2022,Wolves,1,1,15,6,9,4,0,0,L
4,6/8/2022,Nott'm Forest,0,0,5,0,14,1,3,0,L


In [8]:
team_stats_per_match = home_team_stats.append(away_team_stats)

In [9]:
avg_stats_cols = ['total_goals_per_match', 'half_time_goals_per_match', 'shots_per_match', 'shots_on_target_per_match',
            'fouls_per_match', 'corners_per_match', 'yellows_per_match', 'reds_per_match']

team_stats_per_match['Date'] = pd.to_datetime(team_stats_per_match['Date'], format='%d/%m/%Y')

stats = []
for index, row in team_stats_per_match.iterrows():
    team_last5 = team_stats_per_match.loc[(team_stats_per_match['team'] == row['team']) & (team_stats_per_match['Date'] <= row['Date'])].sort_values(by='Date', ascending=False).head(3)
    stats.append(team_last5.iloc[0:3,:].mean(axis=0).values[0:8])
avg_stats = pd.DataFrame(stats, columns=avg_stats_cols)

C:\Users\skinn\AppData\Local\Temp/ipykernel_33200/3585830343.py:9: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  stats.append(team_last5.iloc[0:3,:].mean(axis=0).values[0:8])
C:\Users\skinn\AppData\Local\Temp/ipykernel_33200/3585830343.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stats.append(team_last5.iloc[0:3,:].mean(axis=0).values[0:8])


In [10]:
team_stats_per_match = pd.concat([team_stats_per_match.reset_index(drop=True), avg_stats], axis=1, ignore_index=False)

In [11]:
team_stats_per_match.head()

,Date,team,total_goals,half_time_goals,shots,shots_on_target,fouls,corners,yellows,reds,home_result,away_result,total_goals_per_match,half_time_goals_per_match,shots_per_match,shots_on_target_per_match,fouls_per_match,corners_per_match,yellows_per_match,reds_per_match
0,2022-08-05,Crystal Palace,0,0,10,2,16,3,1,0,L,NaN,0.0,0.0,10.0,2.0,16.0,3.0,1.0,0.0
1,2022-08-06,Fulham,2,1,9,3,7,4,2,0,D,NaN,2.0,1.0,9.0,3.0,7.0,4.0,2.0,0.0
2,2022-08-06,Bournemouth,2,1,7,3,18,5,3,0,W,NaN,2.0,1.0,7.0,3.0,18.0,5.0,3.0,0.0
3,2022-08-06,Leeds,2,1,12,4,13,6,2,0,W,NaN,2.0,1.0,12.0,4.0,13.0,6.0,2.0,0.0
4,2022-08-06,Newcastle,2,0,23,10,9,11,0,0,W,NaN,2.0,0.0,23.0,10.0,9.0,11.0,0.0,0.0


In [12]:
home_team_stats = team_stats_per_match.iloc[:int(team_stats_per_match.shape[0]/2),:]
away_team_stats = team_stats_per_match.iloc[int(team_stats_per_match.shape[0]/2):,:]

home_team_stats = home_team_stats.rename(columns={'team':'name'})
away_team_stats = away_team_stats.rename(columns={'team':'name'})

home_team_stats.columns = ['team_1_' +str(col) for col in home_team_stats.columns]
away_team_stats.columns = ['team_2_' +str(col) for col in away_team_stats.columns]

In [13]:
match_stats = pd.concat([home_team_stats, away_team_stats.reset_index(drop=True)], axis=1, ignore_index=False)
#match_stats = match_stats.drop(match_stats.columns[[11,30]], axis=1).reset_index(drop=True)
match_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   team_1_Date                       370 non-null    datetime64[ns]
 1   team_1_name                       370 non-null    object        
 2   team_1_total_goals                370 non-null    int64         
 3   team_1_half_time_goals            370 non-null    int64         
 4   team_1_shots                      370 non-null    int64         
 5   team_1_shots_on_target            370 non-null    int64         
 6   team_1_fouls                      370 non-null    int64         
 7   team_1_corners                    370 non-null    int64         
 8   team_1_yellows                    370 non-null    int64         
 9   team_1_reds                       370 non-null    int64         
 10  team_1_home_result                370 non-null    

In [14]:
match_stats = match_stats.drop(columns=['team_1_away_result', 'team_2_home_result'])

In [15]:
match_stats['total_goals_per_match_diff'] = (match_stats['team_1_total_goals_per_match'] - match_stats['team_2_total_goals_per_match'])
match_stats['half_time_goals_per_match_diff'] = (match_stats['team_1_half_time_goals_per_match'] - match_stats['team_2_half_time_goals_per_match'])
match_stats['shots_per_match_diff'] = (match_stats['team_1_shots_per_match'] - match_stats['team_2_shots_per_match'])
match_stats['shots_on_target_per_match_diff'] = (match_stats['team_1_shots_on_target_per_match'] - match_stats['team_2_shots_on_target_per_match'])
match_stats['fouls_per_match_diff'] = (match_stats['team_1_fouls_per_match'] - match_stats['team_2_fouls_per_match'])
match_stats['corners_per_match_diff'] = (match_stats['team_1_corners_per_match'] - match_stats['team_2_corners_per_match'])
match_stats['yellows_per_match_diff'] = (match_stats['team_1_yellows_per_match'] - match_stats['team_2_yellows_per_match'])
match_stats['reds_per_match_diff'] = (match_stats['team_1_reds_per_match'] - match_stats['team_2_reds_per_match'])

In [16]:
match_stats['team_1_home_result'] = match_stats[['team_1_home_result']].replace(['W','L','D'],[0,1,2])

In [17]:
match_stats.to_csv('match_stats.csv')

In [18]:
team_stats = {}

for index, row in match_stats.iterrows():
    team_columns = ['team_1_name', 'team_2_name']
    stat_columns = ['total_goals_per_match', 'half_time_goals_per_match', 'shots_per_match',
                   'shots_on_target_per_match', 'fouls_per_match', 'corners_per_match', 'yellows_per_match',
                   'reds_per_match']
    for column in team_columns:
        team_column_name = column.replace('_name', '')
        team = row[column]
        if team not in team_stats:
            team_stats[team] = {}
            
        for stat_column in stat_columns:
            team_column = f'{team_column_name}_{stat_column}'
            stat_field = row[team_column]
            team_stats[team][stat_column] = stat_field



In [19]:
output_file = 'last3.csv'

stat_columns = set()
for team_data in team_stats.values():
    stat_columns.update(team_data.keys())
    
header = ['Team'] + list(stat_columns)


data_rows = []
for team, team_data in team_stats.items():
    data_row = [team]
    for column in stat_columns:
        data_row.append(team_data.get(column, ''))
    data_rows.append(data_row)
    
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(data_rows)

163